# 🧠 Automatic Tumor Detection System
## BME 271D Final Project - Ege, Max, Sasha

### Using Frequency-Domain Analysis for Medical Image Segmentation

**What this does:**
- Analyzes medical images using FFT (Fast Fourier Transform)
- Detects tumors using multiple frequency-domain methods
- Filters out image borders and artifacts
- Provides automatic tumor detection with confidence scores

In [ ]:
# ========== SETUP ==========
!pip install -q numpy matplotlib scipy scikit-image pandas

!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/generate_realistic_tumors.py

!mkdir -p data/images data/masks
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_001.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_002.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_003.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_001.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_002.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_003.png

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, disk, erosion, binary_erosion

def load_image_safe(filepath):
    img = PILImage.open(filepath).convert('L')
    return np.array(img).astype(np.float64) / 255.0

def load_mask_safe(filepath):
    img = PILImage.open(filepath).convert('L')
    return (np.array(img) > 127).astype(np.uint8)

def remove_image_frame_artifacts(mask, border_width=5):
    """
    Remove only the outermost image frame/border artifacts.
    More lenient than clear_border - only removes regions that are
    clearly part of the image border (within 5 pixels of edge).
    """
    h, w = mask.shape
    cleaned = mask.copy()
    
    # Create border mask (just the outer frame)
    border_mask = np.zeros_like(mask, dtype=bool)
    border_mask[:border_width, :] = True
    border_mask[-border_width:, :] = True
    border_mask[:, :border_width] = True
    border_mask[:, -border_width:] = True
    
    # Label all regions
    labeled = label(mask)
    
    # Remove only regions that are >80% within the border zone
    for region in regionprops(labeled):
        region_pixels = (labeled == region.label)
        border_pixels = np.logical_and(region_pixels, border_mask).sum()
        total_pixels = region.area
        
        if border_pixels / total_pixels > 0.8:  # Mostly in border
            cleaned[region_pixels] = False
    
    return cleaned

def detect_tumor_regions(image, sensitivity=0.5):
    """
    Balanced tumor detection:
    - Catches real tumors (bright, round masses)
    - Filters image border artifacts
    - Removes normal tissue boundaries
    """
    h, w = image.shape
    total_pixels = h * w
    
    min_size = int(total_pixels * 0.003)  # 0.3% min
    max_size = int(total_pixels * 0.3)    # 30% max
    
    results = {}
    
    smoothed = ndimage.gaussian_filter(image, sigma=2)
    mean_val = smoothed.mean()
    std_val = smoothed.std()
    
    # === METHOD 1: Bright region detection ===
    # Look for regions significantly brighter than average
    bright_thresh = mean_val + (1.8 - sensitivity * 0.5) * std_val
    bright_mask = smoothed > bright_thresh
    bright_mask = binary_opening(bright_mask, disk(2))
    bright_mask = remove_image_frame_artifacts(bright_mask, border_width=8)
    bright_mask = remove_small_objects(bright_mask, min_size=min_size)
    results['Intensity_Threshold'] = bright_mask
    
    # === METHOD 2: Otsu thresholding ===
    thresh = threshold_otsu(smoothed)
    otsu_mask = smoothed > thresh
    otsu_mask = remove_image_frame_artifacts(otsu_mask, border_width=8)
    otsu_mask = remove_small_objects(otsu_mask, min_size=min_size)
    results['Baseline_Otsu'] = otsu_mask
    
    # === METHOD 3: FFT High-pass ===
    try:
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=25)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        
        hp_thresh = np.percentile(hp_img, 80 - sensitivity * 15)
        hp_mask = hp_img > hp_thresh
        
        # Combine with brightness (tumors are bright AND textured)
        hp_mask = np.logical_and(hp_mask, smoothed > mean_val + 0.5 * std_val)
        hp_mask = remove_image_frame_artifacts(hp_mask, border_width=10)
        hp_mask = binary_closing(hp_mask, disk(4))
        hp_mask = remove_small_objects(hp_mask, min_size=min_size)
        
        results['FFT_HighPass'] = hp_mask
    except:
        results['FFT_HighPass'] = np.zeros_like(image, dtype=bool)
    
    # === METHOD 4: FFT Band-pass ===
    try:
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=10, r2=60)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        
        bp_thresh = np.percentile(bp_img, 75 - sensitivity * 10)
        bp_mask = bp_img > bp_thresh
        
        bp_mask = np.logical_and(bp_mask, smoothed > mean_val + 0.5 * std_val)
        bp_mask = remove_image_frame_artifacts(bp_mask, border_width=10)
        bp_mask = binary_closing(bp_mask, disk(4))
        bp_mask = remove_small_objects(bp_mask, min_size=min_size)
        
        results['FFT_BandPass'] = bp_mask
    except:
        results['FFT_BandPass'] = np.zeros_like(image, dtype=bool)
    
    # === METHOD 5: Combined (balanced voting) ===
    vote_map = (results['Intensity_Threshold'].astype(float) * 1.2 +  # Weight bright regions more
                results['Baseline_Otsu'].astype(float) * 0.8 +
                results['FFT_HighPass'].astype(float) * 1.0 + 
                results['FFT_BandPass'].astype(float) * 1.0)
    
    # Require moderate agreement (1.5-2.5 votes)
    vote_threshold = max(1.5, 2.2 - sensitivity * 0.7)
    combined = vote_map >= vote_threshold
    
    combined = remove_image_frame_artifacts(combined, border_width=8)
    combined = binary_closing(combined, disk(3))
    combined = remove_small_objects(combined, min_size=min_size)
    
    # Filter by shape: remove very elongated regions
    labeled = label(combined)
    filtered = np.zeros_like(combined)
    
    for region in regionprops(labeled):
        # Size check
        if region.area < min_size or region.area > max_size:
            continue
        
        # Shape check: not too elongated (tumors are somewhat round)
        if region.eccentricity > 0.97:  # Very elongated
            continue
        
        # Compactness check
        circularity = 4 * np.pi * region.area / (region.perimeter ** 2 + 1e-8)
        if circularity < 0.12:  # Too thin/irregular
            continue
        
        filtered[labeled == region.label] = True
    
    results['Combined'] = filtered
    return results

def analyze_detection(image, mask):
    """
    Analyze detected regions with balanced criteria.
    """
    h, w = image.shape
    total_pixels = h * w
    tumor_pixels = mask.sum()
    tumor_area_pct = (tumor_pixels / total_pixels) * 100
    
    # Lower threshold - detect smaller tumors
    if tumor_pixels < total_pixels * 0.003:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    coords = np.where(mask)
    if len(coords[0]) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    center = (int(np.mean(coords[0])), int(np.mean(coords[1])))
    
    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)
    
    if len(regions) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    largest = max(regions, key=lambda x: x.area)
    
    # Balanced confidence scoring
    # Size: 1-20% is good range
    if 1 <= tumor_area_pct <= 20:
        size_score = 1.0
    elif tumor_area_pct < 1:
        size_score = tumor_area_pct / 1.0
    else:
        size_score = max(0, 1 - (tumor_area_pct - 20) / 30)
    
    # Shape: somewhat round
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 2.5, 1.0)
    
    # Contrast: bright vs background
    tumor_intensity = image[mask].mean()
    bg_intensity = image[~mask].mean() if (~mask).any() else 0
    contrast = abs(tumor_intensity - bg_intensity)
    contrast_score = min(contrast * 3.5, 1.0)
    
    confidence = (size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4)
    
    # Lower minimum confidence threshold
    if confidence < 0.2:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    return {
        'detected': True,
        'area_percent': tumor_area_pct,
        'center': center,
        'confidence': np.clip(confidence, 0, 1)
    }

image = None
mask = None
print('✅ Balanced tumor detection system ready')
print('   Optimized for real tumors + artifact filtering')

---
## 📁 Load Image

In [ ]:
# ========== OPTION A: UPLOAD ==========
uploaded = files.upload()

if uploaded:
    image = load_image_safe(list(uploaded.keys())[0])
    mask = None
    print(f'✅ Loaded: {list(uploaded.keys())[0]}')
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray')
    plt.title('Your Image')
    plt.axis('off')
    plt.show()
else:
    print('❌ No file uploaded')

In [ ]:
# ========== OPTION B: SAMPLE ==========
tumor_number = 1

tumor_file = f'tumor_{tumor_number:03d}.png'
image = load_image_safe(f'data/images/{tumor_file}')
mask = load_mask_safe(f'data/masks/{tumor_file}')

print(f'✅ Loaded {tumor_file}')

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(image, cmap='gray')
axes[0].set_title('Image')
axes[0].axis('off')
axes[1].imshow(image, cmap='gray')
axes[1].imshow(mask, cmap='Reds', alpha=0.5)
axes[1].set_title('Ground Truth')
axes[1].axis('off')
plt.show()

---
## 🔬 FFT Analysis

In [ ]:
if image is not None:
    F_shift, mag = ts.compute_fft_spectrum(image)
    fig = ts.visualize_frequency_spectrum(image, F_shift)
    plt.show()
else:
    print('⚠️ No image loaded!')

---
## 🎯 TUMOR DETECTION

In [ ]:
if image is not None:
    print('Running detection...\n')
    
    detection_results = detect_tumor_regions(image, sensitivity=0.5)
    methods = ['Baseline_Otsu', 'Intensity_Threshold', 'FFT_HighPass', 'FFT_BandPass', 'Combined']
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    for idx, method in enumerate(methods):
        detected_mask = detection_results[method]
        analysis = analyze_detection(image, detected_mask)
        
        axes[idx+1].imshow(image, cmap='gray')
        axes[idx+1].imshow(detected_mask, cmap='Reds', alpha=0.6)
        
        if analysis['center']:
            row, col = analysis['center']
            axes[idx+1].plot(col, row, 'g+', markersize=15, markeredgewidth=2)
        
        status = f"Area: {analysis['area_percent']:.1f}%" if analysis['detected'] else "Not detected"
        axes[idx+1].set_title(f'{method}\n{status}', fontsize=11, fontweight='bold')
        axes[idx+1].axis('off')
    
    plt.suptitle('Tumor Detection Methods', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary
    print('\n' + '='*70)
    print(f'{"Method":<20} {"Detected":<12} {"Area %":<12} {"Confidence":<12}')
    print('='*70)
    for method in methods:
        analysis = analyze_detection(image, detection_results[method])
        det = '✓ YES' if analysis['detected'] else '✗ No'
        area = f"{analysis['area_percent']:.2f}%"
        conf = f"{analysis['confidence']*100:.0f}%" if analysis['detected'] else '-'
        print(f'{method:<20} {det:<12} {area:<12} {conf:<12}')
    print('='*70)
else:
    print('⚠️ No image!')

---
## 📋 FINAL VERDICT

In [ ]:
if image is not None:
    final_mask = detection_results['Combined']
    final_analysis = analyze_detection(image, final_mask)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    if final_analysis['detected']:
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.6)
        if final_analysis['center']:
            row, col = final_analysis['center']
            axes[1].plot(col, row, 'g+', markersize=25, markeredgewidth=3)
    axes[1].set_title('Detection', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].axis('off')
    
    if final_analysis['detected']:
        if final_analysis['confidence'] > 0.6:
            status = '⚠️ TUMOR DETECTED'
            verdict = '🔴 HIGH CONFIDENCE'
        elif final_analysis['confidence'] > 0.35:
            status = '❓ POSSIBLE TUMOR'
            verdict = '🟡 MEDIUM CONFIDENCE'
        else:
            status = '❓ SUSPICIOUS REGION'
            verdict = '🟠 LOW CONFIDENCE'
    else:
        status = '✅ NO TUMOR DETECTED'
        verdict = '🟢 Normal tissue'
    
    results_text = f"""
    
    {status}
    
    ─────────────────────────────
    
    Confidence: {final_analysis['confidence']*100:.1f}%
    
    Tumor Area: {final_analysis['area_percent']:.2f}%
    
    {verdict}
    """
    
    axes[2].text(0.1, 0.5, results_text, fontsize=14, verticalalignment='center',
                fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    axes[2].set_title('Results', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*50)
    print(f'  {verdict}')
    print('='*50)
else:
    print('⚠️ No image!')

---
## 📊 Validation

In [ ]:
if image is not None and mask is not None:
    detected = detection_results['Combined']
    
    intersection = np.logical_and(detected, mask).sum()
    union = np.logical_or(detected, mask).sum()
    dice = 2 * intersection / (detected.sum() + mask.sum() + 1e-8)
    iou = intersection / (union + 1e-8)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].imshow(mask, cmap='Greens', alpha=0.5)
    axes[0].set_title('Ground Truth', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    axes[1].imshow(detected, cmap='Reds', alpha=0.5)
    axes[1].set_title('Detection', fontweight='bold')
    axes[1].axis('off')
    
    overlap = np.zeros((*image.shape, 3))
    overlap[mask > 0] = [0, 1, 0]
    overlap[detected > 0] = [1, 0, 0]
    overlap[np.logical_and(mask, detected)] = [1, 1, 0]
    
    axes[2].imshow(image, cmap='gray')
    axes[2].imshow(overlap, alpha=0.6)
    axes[2].set_title(f'Overlap (Dice: {dice:.3f})', fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f'\nDice: {dice:.3f}, IoU: {iou:.3f}')
else:
    print('ℹ️ No ground truth for validation')